# Basic SQL Project: Funds & Investments

## Overview
This project contains a set of SQL queries written to analyse a relational database about companies, venture funds, funding rounds, investments, acquisitions, and people/education.

## Purpose
The goal is to answer a range of analytical questions using SQL (filtering, joins, aggregation, and subqueries) and to practise working with a multi-table schema.

## Notebook format
For each question, I provide:
- a short goal statement,
- the SQL query,
- a brief explanation of what the query returns.

## Data notice
The analysis was completed in the Yandex Practicum database environment.
Raw data is not redistributed here; the repository contains SQL queries and methodology only.



## Database schema

The database consists of 7 tables:

- **company** — startup companies (category, status, country_code, founded_at, funding_total, milestones, etc.)
- **fund** — venture funds (country_code, invested_companies, investment_rounds, milestones, etc.)
- **funding_round** — company funding rounds (company_id, funded_at, funding_round_type, raised_amount, is_first_round, is_last_round, etc.)
- **investment** — links funds to company funding rounds (funding_round_id, company_id, fund_id)
- **acquisition** — acquisitions between companies (acquiring_company_id, acquired_company_id, term_code, price_amount, acquired_at)
- **people** — employees linked to companies (company_id, network_username)
- **education** — education records linked to people (person_id, degree_type, institution, graduated_at)

Key relationships:
- `funding_round.company_id → company.id`
- `investment.fund_id → fund.id`, `investment.company_id → company.id`, `investment.funding_round_id → funding_round.id`
- `acquisition.acquiring_company_id → company.id`, `acquisition.acquired_company_id → company.id`
- `people.company_id → company.id`
- `education.person_id → people.id`




## Data quality and preparation (SQL)
Before answering analytical questions, I run a few quick checks to understand data coverage and avoid misleading results.

### Checks performed
- **Row counts** per table to confirm expected data volume.
- **Missing values** in key fields used for joins and grouping (e.g., IDs, country codes, dates, monetary amounts).
- **Duplicates** where a unique record is expected (e.g., IDs or key combinations).
- **Join integrity** (spot-check that key relationships behave as expected: companies ↔ funding rounds, funds ↔ investments, etc.).
- **Filtering rules** used consistently in analysis (e.g., excluding `funding_total = 0` when focusing on funded companies).

> Outputs from these checks are used to set consistent filters and assumptions for the rest of the notebook.


## Section 1 — Basic queries

In this section, I start with simple filters and aggregations to understand key counts and totals.
These queries also confirm that the main fields and categories are used consistently before moving to more complex joins.

### Q1. How many companies have status `closed`?
Goal: count the number of companies with status `closed`.


In [ ]:
SELECT COUNT(id)
FROM company
WHERE status = 'closed';

What this query returns: one number — the count of closed companies in the dataset.

Why it matters: this gives a baseline outcome group that can be compared with other statuses in later sections.

### Q2. What are the funding totals for US news companies?
Goal: list `funding_total` for companies in the USA with category = `news`, ordered from highest to lowest.

In [ ]:
SELECT funding_total
FROM company
WHERE country_code = 'USA'
  AND category_code = 'news'
ORDER BY funding_total DESC;

What this query returns: a ranked list of funding totals for US companies in the `news` category.

Why it matters: this shows the range of funding in one specific segment and helps spot very large values (outliers) early.


### Q3. What is the total value of cash acquisitions in 2011–2013?
Goal: calculate the total `price_amount` for acquisitions with `term_code = 'cash'` that happened between 2011 and 2013.


In [ ]:
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash'
AND EXTRACT(YEAR FROM CAST(acquired_at AS date)) BETWEEN '2011' AND '2013';

What this query returns: one number — the total value of cash-only acquisition deals in 2011–2013.

Why it matters: it provides a simple benchmark for acquisition activity over a defined period.


### Q4. Which people have a `network_username` that starts with "Silver"?
Goal: return first name, last name, and `network_username` for people whose account name begins with `Silver`.


In [ ]:
SELECT first_name,
       last_name,
       network_username
FROM people
WHERE network_username LIKE 'Silver%';

What this query returns: a list of people whose `network_username` starts with “Silver”.

Why it matters: this is a simple example of text-based filtering and can be used to quickly segment records by a naming pattern.


### Q5. Which people have `network_username` containing "money" and a last name starting with "K"?
Goal: return all columns for people whose `network_username` contains `money` and whose last name begins with `K`.

In [ ]:
SELECT *
FROM people
WHERE network_username LIKE '%money%'
AND last_name LIKE 'K%';

What this query returns: all records for people matching both conditions (`money` in `network_username`, and last name starting with “K”).

Why it matters: this shows how to combine multiple filters to define a specific subset of records for further inspection.



### Q6. Which countries have the highest total company funding?
Goal: compute the total `funding_total` by `country_code` and sort countries from highest to lowest.

In [ ]:
SELECT country_code,
       SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC;

What this query returns: total company funding per country (ranked).

Why it matters: it highlights where most funding is concentrated and helps choose meaningful country segments for later analysis.

### Q7. On which dates do funding rounds show real variation in raised amounts?
Goal: for each funding date, return the minimum and maximum `raised_amount`, excluding dates where the minimum is 0 or where min = max.

In [ ]:
SELECT funded_at::date AS round_date,
       MIN(raised_amount),
       MAX(raised_amount)
FROM funding_round
GROUP BY round_date
HAVING MIN(raised_amount)<>0
AND MIN(raised_amount)<>MAX(raised_amount);

What this query returns: dates where funding rounds have non-zero minimum raised amounts and where raised amounts vary (min ≠ max).

Why it matters: it helps spot dates with meaningful differences in round sizes, rather than uniform or zero-only values.


### Q8. How can we categorise funds by investment activity?
Goal: create an activity category for each fund based on the number of companies it invested in (`invested_companies`).
- `high_activity`: 100+ companies
- `middle_activity`: 20–99 companies
- `low_activity`: fewer than 20 companies

In [ ]:
SELECT *,
        CASE
            WHEN invested_companies>=100 THEN 'high_activity'
            WHEN (invested_companies>=20 AND invested_companies<100) THEN 'middle_activity'
            WHEN invested_companies<20 THEN 'low_activity'
        END
FROM fund;

What this query returns: all fund records plus a new field `activity_category` showing each fund’s activity level.

Why it matters: it creates clear segments that can be compared in later analysis (e.g., average number of rounds by activity group).


### Q9. What is the average number of investment rounds by fund activity category?
Goal: for each activity category (from Q8), compute the average number of investment rounds (`investment_rounds`), rounded to the nearest whole number, and sort ascending.


In [ ]:
SELECT
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       ROUND(AVG(investment_rounds)) AS mean
FROM fund
GROUP BY activity
ORDER BY mean;

What this query returns: one row per activity category with the rounded average number of investment rounds.

Why it matters: it compares how actively different types of funds participate across rounds.

### Q10. Which countries have the most active investor funds (founded in 2010–2012)?
Goal: for each country, calculate the minimum, maximum, and average number of companies invested in (`invested_companies`) for funds founded between 2010 and 2012 (inclusive).
Exclude countries where the minimum `invested_companies` is 0.
Return the top 10 countries ranked by the average (descending).



In [ ]:
SELECT country_code,
       AVG(invested_companies),
       MIN(invested_companies),
       MAX(invested_companies)
FROM fund
WHERE EXTRACT(YEAR FROM CAST(founded_at AS timestamp)) BETWEEN '2010' AND '2012'
GROUP BY country_code
HAVING NOT MIN(invested_companies)=0
ORDER BY AVG(invested_companies) DESC
LIMIT 10;

What this query returns: a country-level summary (min/max/avg `invested_companies`) for funds founded in 2010–2012, filtered to exclude countries with zero minimum activity, and ranked to show the top 10 most active investor countries.

Why it matters: it identifies the most active fund geographies based on how many companies their funds typically invest in.



### Q11. List all employees and their education institution (if available)
Goal: return each person’s first and last name, and the education institution they attended (if known).

In [ ]:
SELECT p.first_name,
       p.last_name,
       e.instituition
FROM people AS p
LEFT JOIN education AS e ON p.id = e.person_id;

What this query returns: a list of people, with an institution value where an education record exists (otherwise NULL).

Why it matters: it shows how to enrich people records with optional education data without losing people who have no education entries.


### Q12. Which companies have employees from the largest number of universities?
Goal: for each company, count the number of unique education institutions attended by its employees, then return the top 5 companies.

In [ ]:
SELECT c.name,
       COUNT(DISTINCT e.instituition)
FROM company AS c
JOIN people AS p ON c.id = p.company_id
JOIN education AS e ON p.id = e.person_id
GROUP BY c.name
ORDER BY COUNT(DISTINCT e.instituition) DESC
LIMIT 5;

What this query returns: top 5 companies ranked by the number of unique education institutions among their employees.

Why it matters: it provides a simple company-level diversity indicator based on education records.

### Q13. Which companies raised funds in only one round and are now closed?
Goal: find companies that have exactly one funding round (both first and last) and whose status is `closed`.

In [ ]:
SELECT DISTINCT name
FROM company
WHERE id IN (SELECT company_id
FROM funding_round
WHERE is_first_round=1
AND is_last_round=1)
AND status = 'closed';

What this query returns: closed companies that had a single funding round (first round = last round).

Why it matters: it identifies a specific outcome pattern (one-round funding followed by closure) that can be analysed further.

### Q14. Which employees work at closed companies that had only one funding round?
Goal: return unique employee IDs for people who work at companies that are `closed` and have exactly one funding round (first = last).

In [ ]:
SELECT DISTINCT id
FROM people
WHERE company_id IN (SELECT DISTINCT id
FROM company
WHERE id IN (SELECT company_id
FROM funding_round
WHERE is_first_round=1
AND is_last_round=1)
AND status = 'closed')
;

What this query returns: a distinct list of employee IDs matching the defined company subset.

Why it matters: it defines the employee group used in the next education-related queries.


### Q15. Education institutions for employees from Q14
Goal: return unique pairs of employee and institution for employees from the target group.


In [ ]:
SELECT DISTINCT person_id,
       instituition
FROM education
WHERE person_id IN (SELECT DISTINCT id
                        FROM people
                        WHERE company_id IN (SELECT DISTINCT id
                                                FROM company
                                                WHERE id IN (SELECT company_id
                                                                FROM funding_round
                                                                WHERE is_first_round=1
                                                                AND is_last_round=1)
                                                AND status = 'closed')
     );

What this query returns: unique (person_id, institution) pairs for the selected employees.

Why it matters: it links the employee subset to education data for further counting/averaging.


### Q16. Count education institutions for each employee from Q14
Goal: for each employee ID from Q14, count how many education records they have (including repeated instituitions).


In [ ]:
SELECT DISTINCT person_id as id,
       COUNT(instituition)
                FROM education
                WHERE person_id IN (SELECT DISTINCT id
                                        FROM people
                                        WHERE company_id IN (SELECT DISTINCT id
                                                                FROM company
                                                                WHERE id IN (SELECT company_id
                                                                                FROM funding_round
                                                                                WHERE is_first_round=1
                                                                                AND is_last_round=1)
                                                                AND status = 'closed')
     )
GROUP BY person_id;

What this query returns: one row per employee (person_id) with the number of education records linked to them (including repeated institutions).

Why it matters: this creates a per-person count that is used in the next step to calculate an average for the group.

### Q17. Average number of education institutions per employee (not only unique)
Goal: extend Q16 and calculate the average number of education records per employee for the selected group.
Return a single value.


In [ ]:
SELECT --DISTINCT person_id as id,
       --COUNT(instituition),
       AVG(COUNT(instituition)) OVER()
FROM education
WHERE person_id IN (SELECT DISTINCT id
                    FROM people
                    WHERE company_id IN (SELECT DISTINCT id
                                         FROM company
                                         WHERE id IN (SELECT company_id
                                                      FROM funding_round
                                                      WHERE is_first_round=1
                                                      AND is_last_round=1)
                                         AND status = 'closed')
                     )
GROUP BY person_id
LIMIT 1;

What this query returns: one value — the average number of education records per employee in the selected group.

Why it matters: it summarises the group with a single metric that can be compared with other groups later.


### Q18. What is the average number of education records for Facebook employees?
Goal: calculate the average count of education records per person for employees who work at Facebook.

In [ ]:
SELECT --DISTINCT person_id as id,
       --COUNT(instituition),
       AVG(COUNT(instituition)) OVER()
FROM education
WHERE person_id IN (SELECT DISTINCT id
                    FROM people
                    WHERE company_id IN (SELECT id FROM company WHERE name = 'Facebook')
                     )
GROUP BY person_id
LIMIT 1;

What this query returns: one number — the average number of education records per person for Facebook employees.

Why it matters: it provides a comparison point against the employee group defined in Q17.


### Q19. Which funds invested in companies with 7+ milestones in 2012–2013 rounds?
Goal: build a table with:
- `name_of_fund` — fund name
- `name_of_company` — company name
- `amount` — amount raised in the funding round
Include only companies with more than 6 milestones and rounds funded in 2012–2013.

In [ ]:
SELECT DISTINCT f.name AS name_of_fund,
       c.name AS name_of_company,
       fr.raised_amount AS amount
FROM fund AS f
JOIN investment AS i ON f.id = i.fund_id
JOIN company AS c ON i.company_id = c.id
JOIN funding_round AS fr ON i.funding_round_id = fr.id
WHERE EXTRACT(YEAR FROM CAST(fr.funded_at AS timestamp)) BETWEEN '2012' AND '2013'
AND c.milestones>6;

What this query returns: a table linking funds to companies (with 7+ milestones) and the amount raised in 2012–2013 rounds.

Why it matters: it connects investor activity to more “mature” companies (using milestones as a simple maturity proxy).


### Q20. Acquisition deals: purchase price vs acquired company total funding
Goal: build a table with:
- acquiring company name,
- deal amount (`price_amount`),
- acquired company name,
- acquired company total funding (`funding_total`),
- the rounded ratio (deal amount / `funding_total`).
Exclude deals where the deal amount is 0 and acquired companies where `funding_total` is 0.
Sort by deal amount (descending), then by acquired company name (A–Z), and return the top 10 rows.



In [ ]:
SELECT c1.name,
       a.price_amount,
       c2.name,
       c2.funding_total,
       ROUND(a.price_amount/c2.funding_total)
FROM acquisition AS a
JOIN company AS c1 ON c1.id = a.acquiring_company_id
JOIN company AS c2 ON c2.id = a.acquired_company_id
WHERE NOT a.price_amount=0
AND NOT c2.funding_total=0
ORDER BY a.price_amount DESC, c2.name
LIMIT 10;

What this query returns: the top 10 acquisition deals with a computed ratio comparing purchase price to the acquired company’s `funding_total`.

Why it matters: it provides a simple, comparable metric to rank deals by how large the purchase price is relative to the acquired company’s total funding.



### Q21. Which social companies received non-zero funding in 2010–2013, and in which month?
Goal: return company names in the `social` category that received funding between 2010 and 2013 (inclusive), excluding zero-amount rounds, and show the month number of the funding round.


In [ ]:
SELECT c.name,
       EXTRACT(MONTH FROM fr.funded_at::date)
FROM company AS c
JOIN funding_round AS fr ON c.id = fr.company_id
WHERE c.category_code = 'social'
AND EXTRACT(YEAR FROM fr.funded_at::date) BETWEEN '2010' AND '2013'
AND NOT fr.raised_amount=0;

What this query returns: social-category companies with funded rounds in 2010–2013 and the month each round took place.

Why it matters: it adds a simple time dimension (month) for analysing funding activity patterns within a single category.

### Q22. Monthly summary (2010–2013): US funds investing, acquired companies, and total deal value
Goal: for each month number (1–12) in 2010–2013, return:
- month number,
- number of unique US funds that invested in that month,
- number of companies acquired in that month,
- total value of acquisition deals in that month.

In [ ]:
WITH
    t1 AS (SELECT EXTRACT(MONTH FROM fr.funded_at::date) AS month,
                   COUNT(DISTINCT f.name)
            FROM funding_round AS fr
            JOIN investment AS i ON fr.id = i.funding_round_id
            JOIN fund AS f ON i.fund_id = f.id
            WHERE EXTRACT(YEAR FROM fr.funded_at::date) BETWEEN '2010' AND '2013'
            AND f.country_code='USA'
            GROUP BY month
            ORDER BY month),

     t2 AS (SELECT EXTRACT(MONTH FROM acquired_at::date) AS month,
                   COUNT(id),
                   SUM(price_amount)
            FROM acquisition
            WHERE EXTRACT(YEAR FROM acquired_at::date) BETWEEN '2010' AND '2013'
            GROUP BY month
            ORDER BY month)
SELECT t1.month AS month,
        t1.count AS fund_usa,
        t2.count AS full_acq,
        t2.sum AS sum_amt
FROM t1 FULL JOIN t2 ON t1.month = t2.month

What this query returns: a month-by-month table linking investment activity (US funds) with acquisition activity (count and total deal value).

Why it matters: it provides a compact time-based view that can be used to compare investment and acquisition patterns across months.


### Q23. Country-level funding summary for startups founded in 2011, 2012, and 2013
Goal: build a summary table with the average `funding_total` by country for each year (2011, 2012, 2013) as separate columns.
Include only countries that have startups founded in **all three years**.
Sort by the 2011 average (descending).


In [ ]:
WITH
        year2011 AS (SELECT country_code,
                           AVG(funding_total) as mean11
                    FROM company
                    WHERE EXTRACT(YEAR FROM founded_at::date) = 2011
                    --AND NOT funding_total=0
                    GROUP BY country_code
                    ),
         year2012 AS (SELECT country_code,
                           AVG(funding_total) as mean12
                    FROM company
                    WHERE EXTRACT(YEAR FROM founded_at::date) = 2012
                    --AND NOT funding_total=0
                    GROUP BY country_code
                    ),
          year2013 AS (SELECT country_code,
                           AVG(funding_total) as mean13
                    FROM company
                    WHERE EXTRACT(YEAR FROM founded_at::date) = 2013
                    --AND NOT funding_total=0
                    GROUP BY country_code
                      )
SELECT year2011.country_code,
       year2011.mean11,
       year2012.mean12,
       year2013.mean13
FROM year2011
INNER JOIN year2012 ON year2011.country_code = year2012.country_code
INNER JOIN year2013 ON year2012.country_code = year2013.country_code
ORDER BY year2011.mean11 DESC;


What this query returns: countries present in all three years, with average funding totals shown side-by-side for 2011–2013.

Why it matters: it enables a consistent year-by-year comparison for the same set of countries, avoiding missing-year bias.

## Conclusion
This notebook demonstrates how SQL can be used to analyse venture funding and acquisitions data in a relational database.

Across the queries, I:
- explored baseline coverage (statuses, geography, funding levels),
- compared funding activity across countries, categories, years, and round types,
- segmented funds by activity and summarised investor behaviour,
- linked acquisition prices to prior investment to compare deal “multiples”,
- used people/education tables only in aggregated form to avoid exposing personal details.

The same approach can be extended by adding time series charts or building a dashboard on top of the query outputs.
